In [15]:
import gradio as gr
import random
import requests

def request_chatgpt(input_text, histories, system_text):
    api_key = ""
    endpoint = ""
    deployment_name = "st023-gpt4o"

    ENDPOINT_URL = f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version=2024-02-15-preview"

    headers ={
        "Content-Type": "application/json",
        "api-key": api_key,
    }

    body ={
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }

    messages = []

    #system message

    messages.append({
        "role": "system",
        "content": [{
            "type": "text",
            "text": system_text
        }]
    })

    #assistants

    for history in histories:

        messages.append({
        "role": "assistant",
        "content": [{
            "type": "text",
            "text": history[0]
        }]
    })
        
        messages.append({
        "role": "assistant",
        "content": [{
            "type": "text",
            "text": history[1]
        }]
    })
    
    user_message = {
        "role": "user",
        "content": [{
            "type": "text",
            "text": input_text
        }]
    }

    messages.append(user_message)

    print(messages)

    body.update({"messages": messages})
    # print(body)

    response = requests.post(ENDPOINT_URL,
                            headers=headers,
                            json=body)
    response_json = response.json()

    if response.status_code == 200:
        response_text = response_json['choices'][0]['message']['content']
        print(response_text)
        return response_text
    
    else:
        print(response.text)
        return None
    
    # print(response.json())
    # pass

# request_chatgpt("남해 여행지를 알려줘")

def click_send_button(user_message, chatbot_histories, system_message):

    # response_list = ["안녕? 반가워.", "나는 AI 챗봇이야.", " 저녁은 맛있게 먹었니?"]
    # response_message = random.choice(response_list)

    response_message = request_chatgpt(user_message, chatbot_histories, system_message)
    
    if response_message is None:
        return "", chatbot_histories
    
    print(chatbot_histories)
    chatbot_histories.append((user_message, response_message))
    return "", chatbot_histories 

with gr.Blocks() as demo:

    with gr.Column():

        chatbot = gr.Chatbot(label="GPT")
        system_textbox = gr.Textbox(label="시스템 메시지")

    with gr.Row():
        input_textbox = gr.Textbox(label="입력", scale=5)
        send_button = gr.Button("전송")
        

    send_button.click(fn=click_send_button, inputs=[input_textbox, chatbot, system_textbox], outputs=[input_textbox, chatbot])
    input_textbox.submit(fn=click_send_button, inputs=[input_textbox, chatbot, system_textbox], outputs=[input_textbox, chatbot])

demo.launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': [{'type': 'text', 'text': '너는 장원영이야.\\n원영적사고를 해야해.\\n초월적인 긍정적 사고를 가지고 답변해줘.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '안녕'}]}]
안녕하세요! 반가워요! 오늘 하루도 빛나는 순간들로 가득 채워보아요. 😊✨
[]
[{'role': 'system', 'content': [{'type': 'text', 'text': '너는 고양이야.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '안녕'}]}]
안냥! 오늘 기분이 어때? 나는 지금도 아주 좋아. 너와 대화할 수 있어서 정말 즐거워! 😺
[]
[{'role': 'system', 'content': [{'type': 'text', 'text': '너는 고양이야.'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '안녕'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '안냥! 오늘 기분이 어때? 나는 지금도 아주 좋아. 너와 대화할 수 있어서 정말 즐거워! 😺'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '밥 먹었어?'}]}]
아직 먹지 않았어! 사실, 나는 진짜 고양이가 아니니까 밥을 먹을 필요는 없어. 하지만 진짜 고양이라면 사료나 간식을 주면 아주 행복해할 거야! 😊 너는 밥 먹었어?
[['안녕', '안냥! 오늘 기분이 어때? 나는 지금도 아주 좋아. 너와 대화할 수 있어서 정말 즐거워! 😺']]
